In [1]:
# ============================================================================
# 서울시 범죄 위험 예측기 (Crime Risk Predictor)
# Step 4: 학습된 모델을 활용한 실시간 위험도 예측
# ============================================================================

import pandas as pd
import numpy as np
import joblib
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# 1. 모델 및 인코더 로드
# ============================================================================

class SeoulCrimeRiskPredictor:
    """서울시 범죄 위험도 예측기"""
    
    def __init__(self, model_path, encoder_path):
        """
        초기화: 모델과 인코더 로드
        
        Parameters:
        -----------
        model_path : str
            학습된 RandomForest 모델 경로
        encoder_path : str
            LabelEncoder 딕셔너리 경로
        """
        print("=" * 70)
        print("🚀 서울시 범죄 위험 예측기 초기화")
        print("=" * 70)
        
        # 모델 로드
        self.model = joblib.load(model_path)
        print(f"✅ 모델 로드 완료: {model_path}")
        
        # 인코더 로드
        self.label_encoders = joblib.load(encoder_path)
        print(f"✅ 인코더 로드 완료: {encoder_path}")
        
        # 학습 데이터에서 사용된 확률 분포 정의 (Step 3에서 계산된 값)
        self._init_probability_distributions()
        
        print("\n📋 지원 입력값:")
        print(f"   • 자치구: {len(self.districts)}개")
        print(f"   • 범죄유형: {self.crime_types}")
        print(f"   • 장소카테고리: {self.place_categories}")
        print(f"   • 시간대: {self.time_slots}")
        print("=" * 70)
    
    def _init_probability_distributions(self):
        """학습 데이터 기반 확률 분포 초기화"""
        
        # 자치구 목록
        self.districts = [
            '강남구', '강동구', '강북구', '강서구', '관악구', 
            '광진구', '구로구', '금천구', '노원구', '도봉구',
            '동대문구', '동작구', '마포구', '서대문구', '서초구',
            '성동구', '성북구', '송파구', '양천구', '영등포구',
            '용산구', '은평구', '종로구', '중구', '중랑구'
        ]
        
        # 범죄유형
        self.crime_types = ['살인', '강도', '성폭력', '절도', '폭력']
        
        # 장소카테고리
        self.place_categories = ['주거지구', '업무지구', '다중이용시설', '기타']
        
        # 시간대
        self.time_slots = [
            '00시-03시', '03시-06시', '06시-09시', '09시-12시',
            '12시-15시', '15시-18시', '18시-21시', '21시-24시'
        ]
        
        # 자치구별 범죄 발생 확률 (범죄유형별)
        # Step 3에서 계산된 통계 기반
        self.district_prob = {
            '살인': {'강남구': 0.0447, '강동구': 0.0358, '강북구': 0.0537, '강서구': 0.0418, '관악구': 0.0477,
                    '광진구': 0.0328, '구로구': 0.0388, '금천구': 0.0298, '노원구': 0.0447, '도봉구': 0.0268,
                    '동대문구': 0.0418, '동작구': 0.0328, '마포구': 0.0388, '서대문구': 0.0298, '서초구': 0.0358,
                    '성동구': 0.0298, '성북구': 0.0418, '송파구': 0.0477, '양천구': 0.0328, '영등포구': 0.0537,
                    '용산구': 0.0358, '은평구': 0.0388, '종로구': 0.0418, '중구': 0.0447, '중랑구': 0.0477},
            '강도': {'강남구': 0.0500, '강동구': 0.0389, '강북구': 0.0467, '강서구': 0.0422, '관악구': 0.0533,
                    '광진구': 0.0356, '구로구': 0.0444, '금천구': 0.0311, '노원구': 0.0400, '도봉구': 0.0267,
                    '동대문구': 0.0467, '동작구': 0.0333, '마포구': 0.0489, '서대문구': 0.0311, '서초구': 0.0422,
                    '성동구': 0.0289, '성북구': 0.0400, '송파구': 0.0511, '양천구': 0.0333, '영등포구': 0.0578,
                    '용산구': 0.0400, '은평구': 0.0356, '종로구': 0.0533, '중구': 0.0556, '중랑구': 0.0489},
            '성폭력': {'강남구': 0.0520, '강동구': 0.0398, '강북구': 0.0441, '강서구': 0.0477, '관악구': 0.0563,
                     '광진구': 0.0416, '구로구': 0.0459, '금천구': 0.0355, '노원구': 0.0441, '도봉구': 0.0294,
                     '동대문구': 0.0477, '동작구': 0.0379, '마포구': 0.0502, '서대문구': 0.0343, '서초구': 0.0441,
                     '성동구': 0.0330, '성북구': 0.0416, '송파구': 0.0538, '양천구': 0.0367, '영등포구': 0.0600,
                     '용산구': 0.0416, '은평구': 0.0392, '종로구': 0.0526, '중구': 0.0551, '중랑구': 0.0465},
            '절도': {'강남구': 0.0480, '강동구': 0.0341, '강북구': 0.0299, '강서구': 0.0405, '관악구': 0.0448,
                    '광진구': 0.0363, '구로구': 0.0384, '금천구': 0.0277, '노원구': 0.0363, '도봉구': 0.0235,
                    '동대문구': 0.0405, '동작구': 0.0299, '마포구': 0.0427, '서대문구': 0.0277, '서초구': 0.0405,
                    '성동구': 0.0277, '성북구': 0.0341, '송파구': 0.0469, '양천구': 0.0299, '영등포구': 0.0512,
                    '용산구': 0.0363, '은평구': 0.0320, '종로구': 0.0448, '중구': 0.0491, '중랑구': 0.0363},
            '폭력': {'강남구': 0.0456, '강동구': 0.0349, '강북구': 0.0389, '강서구': 0.0416, '관악구': 0.0483,
                    '광진구': 0.0362, '구로구': 0.0396, '금천구': 0.0282, '노원구': 0.0376, '도봉구': 0.0255,
                    '동대문구': 0.0403, '동작구': 0.0309, '마포구': 0.0429, '서대문구': 0.0295, '서초구': 0.0376,
                    '성동구': 0.0282, '성북구': 0.0362, '송파구': 0.0456, '양천구': 0.0309, '영등포구': 0.0510,
                    '용산구': 0.0349, '은평구': 0.0336, '종로구': 0.0416, '중구': 0.0443, '중랑구': 0.0389}
        }
        
        # 장소별 범죄 발생 확률 (범죄유형별)
        self.place_prob = {
            '살인': {'주거지구': 0.4615, '업무지구': 0.1317, '다중이용시설': 0.2308, '기타': 0.1760},
            '강도': {'주거지구': 0.2532, '업무지구': 0.1570, '다중이용시설': 0.4190, '기타': 0.1708},
            '성폭력': {'주거지구': 0.3876, '업무지구': 0.1810, '다중이용시설': 0.3186, '기타': 0.1138},
            '절도': {'주거지구': 0.2236, '업무지구': 0.1589, '다중이용시설': 0.4830, '기타': 0.1349},
            '폭력': {'주거지구': 0.2774, '업무지구': 0.1527, '다중이용시설': 0.4406, '기타': 0.1280}
        }
        
        # 시간대별 범죄 발생 확률 (범죄유형별)
        self.time_prob = {
            '살인': {'00시-03시': 0.1317, '03시-06시': 0.0991, '06시-09시': 0.0666, '09시-12시': 0.1154,
                    '12시-15시': 0.1361, '15시-18시': 0.1450, '18시-21시': 0.1657, '21시-24시': 0.1405},
            '강도': {'00시-03시': 0.1373, '03시-06시': 0.1059, '06시-09시': 0.0702, '09시-12시': 0.0961,
                    '12시-15시': 0.1210, '15시-18시': 0.1448, '18시-21시': 0.1685, '21시-24시': 0.1545},
            '성폭력': {'00시-03시': 0.1496, '03시-06시': 0.0897, '06시-09시': 0.0608, '09시-12시': 0.1185,
                     '12시-15시': 0.1472, '15시-18시': 0.1617, '18시-21시': 0.2046, '21시-24시': 0.1697},
            '절도': {'00시-03시': 0.0745, '03시-06시': 0.0428, '06시-09시': 0.0609, '09시-12시': 0.1255,
                    '12시-15시': 0.1587, '15시-18시': 0.1895, '18시-21시': 0.2071, '21시-24시': 0.1406},
            '폭력': {'00시-03시': 0.0883, '03시-06시': 0.0425, '06시-09시': 0.0645, '09시-12시': 0.1189,
                    '12시-15시': 0.1571, '15시-18시': 0.1800, '18시-21시': 0.2021, '21시-24시': 0.1457}
        }
        
        # 범죄 심각도
        self.crime_severity = {'살인': 5, '강도': 4, '성폭력': 4, '절도': 1, '폭력': 2}
        
        # 시간대 중앙값
        self.time_to_hour = {
            '00시-03시': 1.5, '03시-06시': 4.5, '06시-09시': 7.5, '09시-12시': 10.5,
            '12시-15시': 13.5, '15시-18시': 16.5, '18시-21시': 19.5, '21시-24시': 22.5
        }
        
        # 심야 시간대
        self.night_times = ['21시-24시', '00시-03시', '03시-06시']
        
        # 출퇴근 시간대
        self.rush_hours = ['06시-09시', '18시-21시']
        
        # 피처 중요도 (모델 해석용)
        self.feature_importance = {
            'p_place': 0.3682,
            'p_district': 0.1408,
            'p_time': 0.1130,
            '장소카테고리': 0.0967,
            'crime_severity': 0.0855,
            '범죄유형': 0.0731,
            '시간_중앙값': 0.0423,
            '시간대': 0.0395,
            '자치구': 0.0255,
            'is_rush_hour': 0.0108,
            'is_night': 0.0046
        }
    
    def _validate_input(self, district, crime_type, place, time_slot):
        """입력값 유효성 검증"""
        errors = []
        
        if district not in self.districts:
            errors.append(f"❌ 자치구 '{district}'는 지원되지 않습니다.")
            errors.append(f"   지원 자치구: {', '.join(self.districts[:5])}...")
        
        if crime_type not in self.crime_types:
            errors.append(f"❌ 범죄유형 '{crime_type}'는 지원되지 않습니다.")
            errors.append(f"   지원 유형: {', '.join(self.crime_types)}")
        
        if place not in self.place_categories:
            errors.append(f"❌ 장소카테고리 '{place}'는 지원되지 않습니다.")
            errors.append(f"   지원 장소: {', '.join(self.place_categories)}")
        
        if time_slot not in self.time_slots:
            errors.append(f"❌ 시간대 '{time_slot}'는 지원되지 않습니다.")
            errors.append(f"   지원 시간대: {', '.join(self.time_slots)}")
        
        return errors
    
    def _prepare_features(self, district, crime_type, place, time_slot):
        """입력값으로부터 모델 피처 생성"""
        
        # 1. Label Encoding
        district_encoded = self.label_encoders['자치구'].transform([district])[0]
        crime_encoded = self.label_encoders['범죄유형'].transform([crime_type])[0]
        place_encoded = self.label_encoders['장소카테고리'].transform([place])[0]
        time_encoded = self.label_encoders['시간대'].transform([time_slot])[0]
        
        # 2. 확률 피처 계산
        p_district = self.district_prob.get(crime_type, {}).get(district, 0.04)
        p_place = self.place_prob.get(crime_type, {}).get(place, 0.25)
        p_time = self.time_prob.get(crime_type, {}).get(time_slot, 0.125)
        
        # 3. 파생 변수 생성
        time_hour = self.time_to_hour.get(time_slot, 12.0)
        is_night = 1 if time_slot in self.night_times else 0
        is_rush_hour = 1 if time_slot in self.rush_hours else 0
        severity = self.crime_severity.get(crime_type, 1)
        
        # 4. 피처 벡터 구성 (학습 시 사용한 순서와 동일)
        features = np.array([[
            district_encoded,    # 자치구_encoded
            crime_encoded,       # 범죄유형_encoded
            place_encoded,       # 장소카테고리_encoded
            time_encoded,        # 시간대_encoded
            p_district,          # p_district
            p_place,             # p_place
            p_time,              # p_time
            time_hour,           # 시간_중앙값
            is_night,            # is_night
            is_rush_hour,        # is_rush_hour
            severity             # crime_severity
        ]])
        
        # 피처 정보 딕셔너리 (해석용)
        feature_info = {
            '자치구': district,
            '범죄유형': crime_type,
            '장소카테고리': place,
            '시간대': time_slot,
            'p_district': p_district,
            'p_place': p_place,
            'p_time': p_time,
            '시간_중앙값': time_hour,
            'is_night': is_night,
            'is_rush_hour': is_rush_hour,
            'crime_severity': severity
        }
        
        return features, feature_info
    
    def _get_risk_level(self, probability):
        """확률에 따른 위험 등급 반환"""
        if probability >= 0.8:
            return "🔴 매우 위험", "CRITICAL"
        elif probability >= 0.6:
            return "🟠 위험", "HIGH"
        elif probability >= 0.4:
            return "🟡 주의", "MODERATE"
        elif probability >= 0.2:
            return "🟢 비교적 안전", "LOW"
        else:
            return "🔵 안전", "SAFE"
    
    def _analyze_risk_factors(self, feature_info):
        """위험 요인 분석"""
        factors = []
        
        # 장소 분석 (가장 중요한 피처)
        if feature_info['p_place'] > 0.4:
            factors.append(f"• 장소 위험도 높음: '{feature_info['장소카테고리']}'는 해당 범죄 다발 지역 (확률: {feature_info['p_place']:.1%})")
        elif feature_info['p_place'] > 0.25:
            factors.append(f"• 장소 위험도 보통: '{feature_info['장소카테고리']}' (확률: {feature_info['p_place']:.1%})")
        
        # 자치구 분석
        if feature_info['p_district'] > 0.05:
            factors.append(f"• 지역 위험도 높음: '{feature_info['자치구']}'는 해당 범죄 다발 자치구 (확률: {feature_info['p_district']:.1%})")
        
        # 시간 분석
        if feature_info['is_night']:
            factors.append(f"• 심야 시간대: '{feature_info['시간대']}'는 범죄 발생 위험이 높은 시간대")
        
        if feature_info['p_time'] > 0.15:
            factors.append(f"• 시간대 위험도 높음: 해당 시간대 범죄 발생 확률 {feature_info['p_time']:.1%}")
        
        # 범죄 심각도
        if feature_info['crime_severity'] >= 4:
            factors.append(f"• 중범죄 유형: '{feature_info['범죄유형']}'은 심각도가 높은 범죄 유형")
        
        return factors if factors else ["• 특별한 위험 요인이 감지되지 않았습니다."]
    
    def predict(self, district, crime_type, place, time_slot, verbose=True):
        """
        범죄 위험도 예측
        
        Parameters:
        -----------
        district : str
            자치구 (예: '마포구')
        crime_type : str
            범죄유형 (예: '폭력')
        place : str
            장소카테고리 (예: '다중이용시설')
        time_slot : str
            시간대 (예: '21시-24시')
        verbose : bool
            상세 출력 여부
            
        Returns:
        --------
        dict : 예측 결과
        """
        
        # 입력값 검증
        errors = self._validate_input(district, crime_type, place, time_slot)
        if errors:
            print("\n".join(errors))
            return None
        
        # 피처 준비
        features, feature_info = self._prepare_features(district, crime_type, place, time_slot)
        
        # 예측
        prediction = self.model.predict(features)[0]
        probability = self.model.predict_proba(features)[0][1]  # 위험(1) 확률
        
        # 위험 등급
        risk_label, risk_code = self._get_risk_level(probability)
        
        # 위험 요인 분석
        risk_factors = self._analyze_risk_factors(feature_info)
        
        # 결과 구성
        result = {
            'input': {
                '자치구': district,
                '범죄유형': crime_type,
                '장소카테고리': place,
                '시간대': time_slot
            },
            'prediction': {
                'is_high_risk': bool(prediction),
                'risk_probability': float(probability),
                'risk_percentage': f"{probability*100:.1f}%",
                'risk_level': risk_label,
                'risk_code': risk_code
            },
            'analysis': {
                'risk_factors': risk_factors,
                'feature_values': feature_info
            }
        }
        
        # 상세 출력
        if verbose:
            self._print_report(result)
        
        return result
    
    def _print_report(self, result):
        """예측 결과 리포트 출력"""
        
        inp = result['input']
        pred = result['prediction']
        analysis = result['analysis']
        
        print("\n")
        print("╔" + "═" * 68 + "╗")
        print("║" + " " * 20 + "🔍 범죄 위험도 예측 리포트" + " " * 21 + "║")
        print("╠" + "═" * 68 + "╣")
        
        # 입력 정보
        print("║ 📋 입력 정보                                                       ║")
        print("╟" + "─" * 68 + "╢")
        print(f"║   • 자치구       : {inp['자치구']:20s}                             ║")
        print(f"║   • 범죄유형     : {inp['범죄유형']:20s}                             ║")
        print(f"║   • 장소카테고리 : {inp['장소카테고리']:20s}                             ║")
        print(f"║   • 시간대       : {inp['시간대']:20s}                             ║")
        
        print("╠" + "═" * 68 + "╣")
        
        # 예측 결과
        print("║ 🎯 예측 결과                                                       ║")
        print("╟" + "─" * 68 + "╢")
        
        # 위험도 시각화 바
        prob = pred['risk_probability']
        bar_length = 40
        filled = int(prob * bar_length)
        bar = "█" * filled + "░" * (bar_length - filled)
        
        print(f"║                                                                    ║")
        print(f"║   위험 지수: [{bar}] {pred['risk_percentage']:>6s}  ║")
        print(f"║                                                                    ║")
        print(f"║   위험 등급: {pred['risk_level']:45s}    ║")
        print(f"║                                                                    ║")
        
        print("╠" + "═" * 68 + "╣")
        
        # 위험 요인 분석
        print("║ 📊 위험 요인 분석                                                  ║")
        print("╟" + "─" * 68 + "╢")
        
        for factor in analysis['risk_factors']:
            # 긴 문자열 처리
            if len(factor) > 64:
                print(f"║   {factor[:61]}...║")
            else:
                print(f"║   {factor:<64s} ║")
        
        print("╠" + "═" * 68 + "╣")
        
        # 권고사항
        print("║ 💡 안전 권고사항                                                   ║")
        print("╟" + "─" * 68 + "╢")
        
        if pred['risk_code'] in ['CRITICAL', 'HIGH']:
            recommendations = [
                "• 가급적 해당 시간대/장소 방문을 자제하세요.",
                "• 외출 시 동행인과 함께 이동하시기 바랍니다.",
                "• 귀중품 휴대를 최소화하세요.",
                "• 비상 연락처(112)를 미리 저장해 두세요."
            ]
        elif pred['risk_code'] == 'MODERATE':
            recommendations = [
                "• 주변 환경에 주의를 기울이세요.",
                "• 어두운 골목보다 큰 길로 이동하세요.",
                "• 늦은 시간에는 대중교통 이용을 권장합니다."
            ]
        else:
            recommendations = [
                "• 기본적인 안전 수칙을 준수하세요.",
                "• 안전한 상황이지만 방심은 금물입니다."
            ]
        
        for rec in recommendations:
            print(f"║   {rec:<64s} ║")
        
        print("╚" + "═" * 68 + "╝")
        print()
    
    def predict_batch(self, scenarios):
        """
        여러 시나리오 일괄 예측
        
        Parameters:
        -----------
        scenarios : list of dict
            각 딕셔너리는 {'자치구', '범죄유형', '장소카테고리', '시간대'} 키를 포함
            
        Returns:
        --------
        pd.DataFrame : 예측 결과 테이블
        """
        results = []
        
        for scenario in scenarios:
            result = self.predict(
                scenario['자치구'],
                scenario['범죄유형'],
                scenario['장소카테고리'],
                scenario['시간대'],
                verbose=False
            )
            
            if result:
                results.append({
                    '자치구': scenario['자치구'],
                    '범죄유형': scenario['범죄유형'],
                    '장소': scenario['장소카테고리'],
                    '시간대': scenario['시간대'],
                    '위험확률': result['prediction']['risk_percentage'],
                    '위험등급': result['prediction']['risk_level']
                })
        
        return pd.DataFrame(results)


# ============================================================================
# 2. 테스트 및 예시 실행
# ============================================================================

if __name__ == "__main__":
    
    # 모델 및 인코더 경로
    MODEL_PATH = '/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/rf_crime_model.pkl'
    ENCODER_PATH = '/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/label_encoders.pkl'
    
    # 예측기 초기화
    predictor = SeoulCrimeRiskPredictor(MODEL_PATH, ENCODER_PATH)
    
    # ========================================================================
    # 테스트 케이스 1: 금요일 밤 11시, 마포구(홍대), 다중이용시설(노상), 폭력
    # ========================================================================
    
    print("\n")
    print("=" * 70)
    print("🧪 테스트 케이스 1: 금요일 밤 홍대 상황")
    print("   (금요일 밤 11시, 마포구 홍대, 다중이용시설/노상, 폭력)")
    print("=" * 70)
    
    result1 = predictor.predict(
        district="마포구",
        crime_type="폭력",
        place="다중이용시설",
        time_slot="21시-24시"
    )
    
    # ========================================================================
    # 테스트 케이스 2: 평일 낮, 강남구, 업무지구, 절도
    # ========================================================================
    
    print("\n")
    print("=" * 70)
    print("🧪 테스트 케이스 2: 평일 낮 강남 오피스 상황")
    print("   (평일 오후 2시, 강남구, 업무지구, 절도)")
    print("=" * 70)
    
    result2 = predictor.predict(
        district="강남구",
        crime_type="절도",
        place="업무지구",
        time_slot="12시-15시"
    )
    
    # ========================================================================
    # 테스트 케이스 3: 새벽, 영등포구, 다중이용시설, 강도
    # ========================================================================
    
    print("\n")
    print("=" * 70)
    print("🧪 테스트 케이스 3: 새벽 영등포 유흥가 상황")
    print("   (새벽 2시, 영등포구, 다중이용시설, 강도)")
    print("=" * 70)
    
    result3 = predictor.predict(
        district="영등포구",
        crime_type="강도",
        place="다중이용시설",
        time_slot="00시-03시"
    )
    
    # ========================================================================
    # 테스트 케이스 4: 주거지역 안전 시나리오
    # ========================================================================
    
    print("\n")
    print("=" * 70)
    print("🧪 테스트 케이스 4: 주간 주거지역 상황")
    print("   (오전 10시, 도봉구, 주거지구, 절도)")
    print("=" * 70)
    
    result4 = predictor.predict(
        district="도봉구",
        crime_type="절도",
        place="주거지구",
        time_slot="09시-12시"
    )
    
    # ========================================================================
    # 일괄 예측 테스트
    # ========================================================================
    
    print("\n")
    print("=" * 70)
    print("📊 다중 시나리오 일괄 예측")
    print("=" * 70)
    
    scenarios = [
        {'자치구': '마포구', '범죄유형': '폭력', '장소카테고리': '다중이용시설', '시간대': '21시-24시'},
        {'자치구': '강남구', '범죄유형': '절도', '장소카테고리': '다중이용시설', '시간대': '18시-21시'},
        {'자치구': '종로구', '범죄유형': '성폭력', '장소카테고리': '주거지구', '시간대': '00시-03시'},
        {'자치구': '영등포구', '범죄유형': '강도', '장소카테고리': '다중이용시설', '시간대': '03시-06시'},
        {'자치구': '서초구', '범죄유형': '폭력', '장소카테고리': '업무지구', '시간대': '15시-18시'},
    ]
    
    batch_results = predictor.predict_batch(scenarios)
    print("\n📋 일괄 예측 결과:")
    print("-" * 70)
    print(batch_results.to_string(index=False))
    
    # 결과 저장
    batch_results.to_csv('/Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/prediction_results.csv', 
                         index=False, encoding='utf-8-sig')
    print("\n💾 예측 결과 저장: prediction_results.csv")
    
    print("\n" + "=" * 70)
    print("✅ 모든 테스트 완료!")
    print("=" * 70)

🚀 서울시 범죄 위험 예측기 초기화
✅ 모델 로드 완료: /Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/rf_crime_model.pkl
✅ 인코더 로드 완료: /Users/keumyoung/Desktop/luckyseven_crime-data-analysis/final_output/label_encoders.pkl

📋 지원 입력값:
   • 자치구: 25개
   • 범죄유형: ['살인', '강도', '성폭력', '절도', '폭력']
   • 장소카테고리: ['주거지구', '업무지구', '다중이용시설', '기타']
   • 시간대: ['00시-03시', '03시-06시', '06시-09시', '09시-12시', '12시-15시', '15시-18시', '18시-21시', '21시-24시']


🧪 테스트 케이스 1: 금요일 밤 홍대 상황
   (금요일 밤 11시, 마포구 홍대, 다중이용시설/노상, 폭력)


╔════════════════════════════════════════════════════════════════════╗
║                    🔍 범죄 위험도 예측 리포트                     ║
╠════════════════════════════════════════════════════════════════════╣
║ 📋 입력 정보                                                       ║
╟────────────────────────────────────────────────────────────────────╢
║   • 자치구       : 마포구                                              ║
║   • 범죄유형     : 폭력                                               ║
║   • 장소카테고리 : 다중이용시설   